In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("..")
from plot_OpSims import plot_OpSims_hist

In [ ]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [ ]:
# import convenience functions
import sys 
sys.path.append("../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory.

In [ ]:
NSIDE = 64
your_username = "rjassef"
folder_mafoutput = "DDFs_EM5_depths_{0:d}".format(NSIDE)
resultDbPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [ ]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [ ]:
#Read the list of current OpSims to be used for the analysis. 
Current_OpSims = np.genfromtxt("../Run_Info/Current_list_of_OpSims.txt", dtype='U')

In [ ]:
#Find if any current OpSims are missing from the runs. 
print(np.setdiff1d(Current_OpSims, list(resultDbs.keys())))

In [ ]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    if runName in Current_OpSims:
        bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

In [ ]:
dbRuns = list(bundleDicts_raw.keys())
print(bundleDicts_raw[dbRuns[0]].keys())
#print(bundleDicts_raw[dbRuns[1]].keys())

In [ ]:
#Make sure that the plots folder exists.
plots_folder = "plots_current_opsims_{0:d}".format(NSIDE)
if not os.path.exists(plots_folder):
    os.mkdir(plots_folder)

### Function to pull out a specific DDF from the bundleDicts 

In [ ]:
def get_bundleDicts_use(filter, DDF_name, bundleDicts_raw):
    bundleDicts_use = dict()
    metricName = 'ExgalM5_with_cuts_AGN_{0}_{1}'.format(DDF_name, filter)
    Key = (1, metricName)
    for run in dbRuns:
        for raw_key in bundleDicts_raw[run].keys():
            if raw_key[1]==metricName:
                #mask = bundleDicts_raw[run][raw_key].metricValues.mask
                #data = bundleDicts_raw[run][raw_key].metricValues.data[~mask]
                #data = data[~(np.isnan(data) | np.isinf(data))]
                bundleDicts_use[run] = dict()
                bundleDicts_use[run][Key] = bundleDicts_raw[run][raw_key]
                break
    return Key, bundleDicts_use

### u-band 3 sigma 

In [ ]:
def mlim_3sigma(mlim_5sigma):
    return mlim_5sigma + 2.5*np.log10(5./3.)

In [ ]:
filter = "u"
#DDF = "AllDDFs"
DDFs = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']

for DDF in DDFs:

    #Create the bundleDict appropriate for this DDF.
    Key = tuple()
    bundleDicts_use = dict()
    Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

    #Set here the useful parameters for the plot.
    data_func = mlim_3sigma
    xlabel = r"u-band 3$\sigma$ depth (mag)"
    title = DDF
    healpix_pixarea = bundleDicts_use[dbRuns[0]][Key].slicer.pixArea * u.sr

    #Make the plot
    plot_OpSims_hist(Key, bundleDicts_use, data_func=data_func, figsize=(10,11), 
                     healpix_pixarea=healpix_pixarea, datamin=None, datamax=None, 
                     color_map=[mpl.cm.summer,mpl.cm.cool], xlabel=xlabel, title=title)

    #Save the plot.
    fname = "{0}/uband_3sigma_depth_{1}.png".format(plots_folder,DDF)
    plt.savefig(fname, dpi=200, bbox_inches='tight')

### g-band 5 sigma

In [ ]:
filter = "g"
#DDF = "AllDDFs"
DDFs = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']

for DDF in DDFs:

    #Create the bundleDict appropriate for this DDF.
    Key = tuple()
    bundleDicts_use = dict()
    Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

    #Set here the useful parameters for the plot.
    data_func = None
    xlabel = r"g-band 5$\sigma$ depth (mag)"
    title = DDF
    healpix_pixarea = bundleDicts_use[dbRuns[0]][Key].slicer.pixArea * u.sr

    #Make the plot
    plot_OpSims_hist(Key, bundleDicts_use, data_func=data_func, figsize=(10,11), 
                     healpix_pixarea=healpix_pixarea, datamin=None, datamax=None, 
                     color_map=[mpl.cm.summer,mpl.cm.cool], xlabel=xlabel, title=title)

    #Save the plot.
    fname = "{0}/gband_3sigma_depth_{1}.png".format(plots_folder,DDF)
    plt.savefig(fname, dpi=200, bbox_inches='tight')

## Plot as a fuction of L* at z=2.5

In [ ]:
#vanden Berk et al. (2001)
data = np.loadtxt("../mstar_estimates/mstar_z.vandenberk.dat")

#Assef et al. (2010)
#data = np.loadtxt("../mstar_estimates/mstar_z.lrt.dat")

mstar = dict()
filters = ['u','g','r','i','z','y']
for k, filter in enumerate(filters):
    mstar[filter] = np.interp([2.5], data[:,0], data[:,k+1])[0]
    print("{0:s} {1:7.3f}".format(filter, mstar[filter]))

### u-band 3-sigma

In [ ]:
def log_QLF_Lstar_frac_u(mlim_5sigma):
    #First, transform to 3 sigma
    mlim_3sigma = mlim_5sigma + 2.5*np.log10(5./3.)
    #Now, transform to Lstar fraction.
    return -0.4*(mlim_3sigma - mstar['u'])

In [ ]:
filter = "u"
#DDF = "AllDDFs"
DDFs = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']

for DDF in DDFs:

    #Create the bundleDict appropriate for this DDF.
    Key = tuple()
    bundleDicts_use = dict()
    Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

    #Set here the useful parameters for the plot.
    data_func = log_QLF_Lstar_frac_u
    xlabel = r"$\log L/L_*(z=2.5)$ for u-band 3$\sigma$ depth"
    title = DDF
    healpix_pixarea = bundleDicts_use[dbRuns[0]][Key].slicer.pixArea * u.sr

    #Make the plot
    plot_OpSims_hist(Key, bundleDicts_use, data_func=data_func, figsize=(10,11), 
                     healpix_pixarea=healpix_pixarea, datamin=None, datamax=None, 
                     color_map=[mpl.cm.summer,mpl.cm.cool], xlabel=xlabel, title=title)

    #Save the plot.
    fname = "{0}/uband_3sigma_L_Lstar_{1}.png".format(plots_folder,DDF)
    plt.savefig(fname, dpi=200, bbox_inches='tight')

### g-band 5-sgima

In [ ]:
def log_QLF_Lstar_frac_g(mlim_5sigma):
    return -0.4*(mlim_5sigma - mstar['g'])

In [ ]:
filter = "g"
#DDF = "AllDDFs"
DDFs = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']

for DDF in DDFs:

    #Create the bundleDict appropriate for this DDF.
    Key = tuple()
    bundleDicts_use = dict()
    Key, bundleDicts_use = get_bundleDicts_use(filter, DDF, bundleDicts_raw)

    #Set here the useful parameters for the plot.
    data_func = log_QLF_Lstar_frac_g
    xlabel = r"$\log L/L_*(z=2.5)$ for g-band 5$\sigma$ depth"
    title = DDF
    healpix_pixarea = bundleDicts_use[dbRuns[0]][Key].slicer.pixArea * u.sr

    #Make the plot
    plot_OpSims_hist(Key, bundleDicts_use, data_func=data_func, figsize=(10,11), 
                     healpix_pixarea=healpix_pixarea, datamin=None, datamax=None, 
                     color_map=[mpl.cm.summer,mpl.cm.cool], xlabel=xlabel, title=title)

    #Save the plot.
    fname = "{0}/gband_3sigma_L_Lstar_{1}.png".format(plots_folder,DDF)
    plt.savefig(fname, dpi=200, bbox_inches='tight')